In [5]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/')

In [6]:
import torch
import pandas as pd
import numpy as np
import statistics
import torch
import random
import time
import numpy as np
#from transformers import AdamW
from torch.utils.tensorboard import SummaryWriter
import pickle
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import shutil
import itertools

from model.GRU_model import *
from pipeline_config import *
from utils import *

In [7]:
# Seeds for comparisons:

torch.manual_seed(1)
np.random.seed(2)
random.seed(3)
torch.use_deterministic_algorithms(True)


In [8]:
with open(train_dataset_path, 'rb') as f:
    train_dataset_dict = pickle.load(f)
    
with open(test_dataset_path, 'rb') as f:
    test_dataset_dict = pickle.load(f)

with open(category_id_to_category_name_path, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)
    
with open(categories_per_indent_path, 'rb') as f:
    categories_per_indent_dict = pickle.load(f)

with open(son_parent_path, 'rb') as f:
    son_parent_dict = pickle.load(f)

In [9]:
print(f'Features: {Features}')
print(f'HiddenSize: {HiddenSize}')
print(f'LayersDim: {LayersDim}')
print(f'OutputDim: {OutputDim}')
print(f'DropoutProb: {DropoutProb}')
print(f'Lr : {Lr}')
print(f'Epochs : {Epochs}')

Features: 1
HiddenSize: 32
LayersDim: 1
OutputDim: 1
DropoutProb: 0.0
Lr : 0.05
Epochs : 250


In [6]:
cnt = 0
for indent in sorted(list(categories_per_indent_dict.keys())):
    for category in categories_per_indent_dict[indent]:
        cnt+=1

cnt

350

In [7]:
def hgru_model(son_parent_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weights_path, seed):
    hgru_models = {}

    categories_counter = 0

    for indent in sorted(list(categories_per_indent_dict.keys())):
        for category in categories_per_indent_dict[indent]:
            print(f'indent: {indent}')
            category_name = category_id_to_name_dict[category]
            print(f'category_name: {category_name}')

            categories_counter+= 1
            print(f'categories_counter: {categories_counter}')

            if int(indent) == 0 or son_parent_dict[category] not in categories_per_indent_dict[indent-1]:
                print(f'category with 0 loss coef: {category_name}|{category}')
                loss_coef=0
                parent_weights=0
            else:
                son = category
                parent = son_parent_dict[son]
                parent_name = category_id_to_name_dict[parent]
                loss_coef = 0.05
                parent_model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb, seed=seed)
                parent_optimizer = Optimizer
                parent_path = weights_path+'_'+str(seed)+'/'+parent_name+'.pt'
                parent_model, optimizer, checkpoint, valid_loss_min = load_checkpoint(parent_path, parent_model, parent_optimizer)
                parent_weights = unify_model_weights(parent_model)
 
            
            train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category_name], test_dataset_dict[category_name])
            print(f'seed: {seed}')
            model=GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb, seed=seed)
            optimizer = Optimizer
            model.to(Device)
            saving_param_path = weights_path+'_'+str(seed)+'/'+category_name+'.pt'
            hgru_models[category_name] = training_and_evaluation(model, train_dataloader, test_dataloader, optimizer, category_name, parent_weights, loss_coef, path=saving_param_path)
                
        
    return hgru_models


In [8]:
hgru_models_2 = hgru_model(son_parent_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weightspath, seed=2)
hgru_models_9 = hgru_model(son_parent_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weightspath, seed=9)
hgru_models_13 = hgru_model(son_parent_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weightspath, seed=13)
hgru_models_26 = hgru_model(son_parent_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weightspath, seed=26)
hgru_models_29 = hgru_model(son_parent_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weightspath, seed=29)
hgru_models_63 = hgru_model(son_parent_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weightspath, seed=63)

indent: 0.0
category_name: All items
categories_counter: 1
category with 0 loss coef: All items|8106.0
seed: 2
indent: 1.0
category_name: All items less energy
categories_counter: 2
seed: 2
indent: 1.0
category_name: All items less food
categories_counter: 3
seed: 2
indent: 1.0
category_name: All items less food and energy
categories_counter: 4
seed: 2
indent: 1.0
category_name: All items less food and shelter
categories_counter: 5
seed: 2
indent: 1.0
category_name: All items less food, shelter, and energy
categories_counter: 6
seed: 2
indent: 1.0
category_name: All items less food, shelter, energy, and used cars and trucks
categories_counter: 7
seed: 2
indent: 1.0
category_name: All items less medical care
categories_counter: 8
seed: 2
indent: 1.0
category_name: All items less shelter
categories_counter: 9
seed: 2
indent: 1.0
category_name: Apparel less footwear
categories_counter: 10
seed: 2
indent: 1.0
category_name: Commodities
categories_counter: 11
seed: 2
indent: 1.0
category_na

Save prediction dict:

In [10]:
categories_lists = list(categories_per_indent_dict.values())
categories_id = list(itertools.chain.from_iterable(categories_lists))
categories = []
for category_id in categories_id:
    categories.append(category_id_to_name_dict[category_id])
    
predictions_dict_2 = get_results_on_test_set(weightspath, train_dataset_dict, test_dataset_dict, categories = categories, seed = 2)
predictions_dict_9 = get_results_on_test_set(weightspath, train_dataset_dict, test_dataset_dict, categories = categories, seed = 9)
predictions_dict_13 = get_results_on_test_set(weightspath, train_dataset_dict, test_dataset_dict, categories = categories, seed = 13)
predictions_dict_26 = get_results_on_test_set(weightspath, train_dataset_dict, test_dataset_dict, categories = categories, seed = 26)
predictions_dict_29 = get_results_on_test_set(weightspath, train_dataset_dict, test_dataset_dict, categories = categories, seed = 29)
predictions_dict_63 = get_results_on_test_set(weightspath, train_dataset_dict, test_dataset_dict, categories = categories, seed = 63)

Admission to movies, theaters, and concerts
Bacon, breakfast sausage, and related products
Bread other than white
Canned fruits
Canned vegetables
Checking account and other bank services
Chicken
Cookies
Crackers, bread, and cracker products
Dried beans, peas, and lentils
Frankfurters
Fresh cakes and cupcakes
Fresh fish and seafood
Fresh sweetrolls, coffeecakes, doughnuts
Frozen and refrigerated bakery products, pies, tarts, turnovers
Frozen vegetables
Ham
Intracity mass transit
Lunchmeats
Oranges, including tangerines
Other condiments
Other pork including roasts and picnics
Other uncooked poultry including turkey
Parking fees and tolls
Pork chops
Prepared salads
Processed fish and seafood
Salt and other seasonings and spices
Sauces and gravies
Ship fare
Uncooked beef roasts
Uncooked beef steaks
Uncooked ground beef
Uncooked other beef and veal
Video discs and other media
White bread
Admissions
Airline fare
Apparel services other than laundry and dry cleaning
Apples
Bananas
Beef and vea

In [11]:
prediction_dicts = [predictions_dict_2, predictions_dict_9, predictions_dict_13, predictions_dict_26, predictions_dict_29, predictions_dict_63]

In [12]:
prediction_dicts

[{'Admission to movies, theaters, and concerts': tensor([[0.1127, 0.1103, 0.1344, 0.1617, 0.1008, 0.2091, 0.2961, 0.0873, 0.1442,
           0.1131, 0.0961, 0.1142, 0.1049, 0.0921, 0.0991, 0.1408, 0.1312, 0.0737,
           0.0885, 0.1638, 0.1267, 0.1054, 0.0829, 0.0901, 0.0916, 0.1075, 0.0625,
           0.0808, 0.1078, 0.0801, 0.0962, 0.1618]]),
  'Bacon, breakfast sausage, and related products': tensor([[ 0.1619,  0.1819,  0.1582,  0.0900,  0.0926,  0.0268,  0.2585,  0.2298,
            0.0985,  0.0258,  0.1085,  0.1168,  0.2115,  0.0724, -0.0065, -0.0335,
            0.0229, -0.0102,  0.0088, -0.0502, -0.0043,  0.0090,  0.0681,  0.1623,
            0.1695,  0.0472,  0.0663,  0.0743,  0.1648,  0.1964,  0.1066,  0.1506]]),
  'Bread other than white': tensor([[ 0.1505,  0.0710,  0.0738,  0.0542,  0.2020,  0.1443,  0.1116,  0.1161,
            0.0982,  0.1253,  0.1635,  0.1339,  0.1737,  0.1579,  0.1307,  0.0935,
            0.1065,  0.1425,  0.0685,  0.0829,  0.0468,  0.0552,  0.0671,

In [13]:
categories = list(prediction_dicts[0].keys())

In [14]:
aggregate_predictions_dict = aggregate_predictions(prediction_dicts, categories)

In [15]:
with open(test_predictions_path, 'wb') as handle:
    pickle.dump(aggregate_predictions_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)